In [ ]:
import json
import os
import sys
from functools import lru_cache
from pathlib import Path

import pandas as pd
from tqdm import tqdm

sys.path.insert(0, "..")
from twittertools.geocode import (
    load_friends_data, load_city_data, load_state_data,
    geocode_simplemaps, test,
)

## Geocode with SimpleMaps

In [ ]:
data = load_friends_data()
data_coded = []
for screen_name, location in tqdm(data):
    res = geocode_simplemaps(location)
    city, country = lookup_index(res)  # could return (None, None)
    data_coded.append(
        dict(screen_name=screen_name, location=location, city=city, country=country)
    )
return pd.DataFrame(data_coded)

### Compute Scores

In [16]:
SCORES = []

numerator = len(res_df[~res_df['country'].isna()])
denomiator = len(res_df['country'])

# without nas
res_df_dropna = res_df[res_df['location'] != ""]
numerator2 = len(res_df_dropna[~res_df_dropna['country'].isna()])
denomiator2 = len(res_df_dropna['country'])

print(f"Success: {numerator}/{denomiator} = {numerator / denomiator}")

print("\nExcluding empty:")
print(f"Success: {numerator2}/{denomiator2} = {numerator2 / denomiator2}")

SCORES.append(numerator2 / denomiator2)

Success: 917/1884 = 0.4867303609341826

Excluding empty:
Success: 917/1482 = 0.6187584345479082


In [29]:
df_cities[df_cities['city'].str.contains("st.", regex=False)]

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
385,st. louis,st. louis,38.6358,-90.2451,united states,us,usa,missouri,NaN,2024074.0,1840001651
1463,port st. lucie,port st. lucie,27.2796,-80.3883,united states,us,usa,florida,NaN,461152.0,1840015119
1970,st. catharines,st. catharines,43.1833,-79.2333,canada,ca,can,ontario,NaN,309319.0,1124140229
1975,st. paul,st. paul,44.9477,-93.1040,united states,us,usa,minnesota,admin,308096.0,1840008940
2194,st. petersburg,st. petersburg,27.7931,-82.6652,united states,us,usa,florida,NaN,265351.0,1840015977
...,...,...,...,...,...,...,...,...,...,...,...
25381,west st. paul,west st. paul,50.0119,-97.1150,canada,ca,can,manitoba,NaN,5368.0,1124001136
25393,st. paul park,st. paul park,44.8360,-92.9949,united states,us,usa,minnesota,NaN,5362.0,1840008917
25512,st. clair,st. clair,42.8262,-82.4930,united states,us,usa,michigan,NaN,5279.0,1840003954
25688,st. james,st. james,33.9467,-78.1132,united states,us,usa,north carolina,NaN,5141.0,1840017954


In [12]:
missed = res_df_dropna[res_df_dropna['country'].isna()]

In [13]:
missed.sample(30)

,screen_name,location,city,country
4264,Pinboard,Lonely street of broken dreams,None,None
2129,nowthisnews,The Internet & NYC,None,None
812,RacismDog,United States,None,None
3003,LeonYin,www,None,None
2481,DigitalBlkHippy,a mac around a bunch of pcs,None,None
3437,glasnt,Eora Country,None,None
3686,Esquiring,NYC // SF // 🌊,None,None
2407,css,"Chris in Bend, OR + Remote Team",None,None
1150,staceyabrams,Georgia,None,None
2828,salimismail,"ÜT: 37.458282,-122.164066",None,None


In [110]:
# with pd.option_context('display.max_rows', None):
#     display(missed)

In [111]:
# with pd.option_context('display.max_rows', None):
#     display(res_df_dropna[~res_df_dropna['country'].isna()])

## GeoCoding with Web

In [1]:
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="twitter-tools")

In [ ]:
df = pd.DataFrame(data_coded)

In [ ]:
def get_country(loc):
    try:
        return loc.split(',')[-1].strip()
    except:
        return None

In [ ]:
s = df['location_address']
df['country'] = s.apply(get_country)

In [ ]:
df.to_csv('data.csv')

In [ ]:
geocoded = {}
for item in data_coded:
    geocoded[item['location_raw']] = dict(
        loc_address=item['location_address'],
        loc_latitude=item['location_latitude'],
        loc_longitude=item['location_longitude']
    )

In [ ]:
with open('geo_cache.json', 'w') as f:
    json.dump(geocoded, f)

In [ ]:
def geocode(loc):
    location = geolocator.geocode(loc)
    if location:
        return dict(
            loc_address=location.address,
            loc_latitude=location.latitude,
            loc_longitude=location.longitude
        )
    return dict(
            loc_address=None,
            loc_latitude=None,
            loc_longitude=None
        )

In [ ]:
df_cities = pd.read_csv('../data/simplemaps/worldcities.csv')

In [ ]:
data_coded = []
for screen_name, location in data:
    country = geocode_simplemaps(location)
    data_coded.append(dict(
        screen_name=screen_name,
        location=location,
        country=country
    ))

In [ ]:
pd.DataFrame(data_coded).country.unique()

In [ ]:
df['location'].apply(geocode_simplemaps)

In [ ]:
data_coded = []
for screen_name, loc in tqdm(data):
    entry = dict(screen_name=screen_name, location_raw=loc)
    # check cache
    if loc in geocoded:
        geo_vals = geocoded[loc]
    else:
        geo_vals = geocode(loc)
    data_coded.append({**entry, **geo_vals})

### GeoJSON

In [ ]:
with open('../data/worldgeo.json') as f:
    geojson = json.load(f)

In [ ]:
[x['properties']['sovereignt'] for x in geojson['features']]